In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential
from keras import Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

2023-10-09 12:24:03.639853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 12:24:04.899482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
state0=np.load('../Data/new/2gsps/state0.npy')
state1=np.load('../Data/new/2gsps/state1.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

In [ ]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

In [ ]:

st,ed=1400,4400
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

x_train=np.mean(x_train,axis=1)
x_test=np.mean(x_test,axis=1)

# x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
print(x_train.shape)

## Logistic Regression

In [ ]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

## SVM

In [ ]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))

## GMM

In [ ]:
Y=list(zip(np.append(np.mean(state0[:,2000:3000,0],axis=1),np.mean(state1[:,2000:3000,0],axis=1)),np.append(np.mean(state0[:,2000:3000,1],axis=1),np.mean(state1[:,2000:3000,1],axis=1))))
gmm_Y=GaussianMixture(n_components=2,covariance_type='full').fit(Y)
bitstring=np.split(gmm_Y.predict(Y),2)
print('P(0|0)=',len(bitstring[0][bitstring[0]==0])/len(bitstring[0]))
print('P(1|1)=',len(bitstring[1][bitstring[1]==1])/len(bitstring[1]))
X=list(zip(np.mean(state0[:,2000:3000,0],axis=1),np.mean(state0[:,2000:3000,1],axis=1)))
gmm_Y_1=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)
X=list(zip(np.mean(state1[:,2000:3000,0],axis=1),np.mean(state1[:,2000:3000,1],axis=1)))
gmm_Y_2=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)

## NN

In [ ]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2)))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16,activation='relu'))
    model.add(Dropout(0.5))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(32,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model
arch().summary()

In [ ]:
import keras.backend as K

def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [1]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# !which python
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-10-09 12:24:19.491512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 12:24:20.668551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-09 12:24:22.842218: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1302281724726980669
 xla_global_id: -1]

In [ ]:
H=model.fit(x_train,y_train,
          validation_data=(x_test, y_test),
          epochs=100,batch_size=16,
          callbacks=callbacks_list,
          verbose=1)

In [ ]:
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i,0]<0.5:
            count1+=1
print(count,count1)

## GDA

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])

        self.means = []
        self.cov_matrices = []
        for c in self.classes:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            cov_matrix_c = np.cov(X_c, rowvar=False)
            self.means.append(mean_c)
            self.cov_matrices.append(cov_matrix_c)

    def predict(self, X):
        posteriors = []

        for c in self.classes:
            class_prior = self.class_priors[int(c)]
            mean = self.means[int(c)]
            cov_matrix = self.cov_matrices[int(c)]
            mvn = multivariate_normal(mean=mean, cov=cov_matrix)
            posterior = class_prior * mvn.pdf(X)
            posteriors.append(posterior)

        posteriors = np.array(posteriors).T
        predicted_labels = np.argmax(posteriors, axis=1)

        return predicted_labels



# Initialize and train GDA
gda = GaussianDiscriminantAnalysis()
gda.fit(x_train, y_train)

# Make predictions
y_pred = gda.predict(x_train)
# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")